In [15]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
import nltk
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import re

In [16]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
df = pd.read_excel(r'./20.xlsx')
df

,순위,서명,저자,출판사,출판년도,권,ISBN,ISBN부가기호,KDC,대출건수,DESCRIPTION
0,1,나미야 잡화점의 기적 :히가시노 게이고 장편소설,지은이: 히가시노 게이고 ;옮긴이: 양윤옥,현대문학,2012,NaN,9788972756194,3830.0,833.600,"33,643",2012년 12월 19일 국내 번역 출간된 이래 6년 연속 베스트셀러 순위 상위권을...
1,2,채식주의자:한강 연작소설,한강,창비,2007,NaN,9788936433598,3810.0,813.600,"24,482",2016년 맨부커상 수상작. 10년 전 작가 한강은 '내 여자의 열매'라는 단편소설...
2,3,82년생 김지영 :조남주 장편소설,지은이: 조남주,민음사,2016,NaN,9788937473135,4810.0,813.700,"21,163",오늘의 젊은 작가 13권. 조남주 장편소설. 시사 교양 프로그램에서 10년 동안 일...
3,4,소년이 온다 :한강 장편소설,지은이: 한강,창비,2014,NaN,9788936434120,3810.0,813.700,"20,696",섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 ...
4,5,미움받을 용기 :자유롭고 행복한 삶을 위한 아들러의 가르침,"기시미 이치로,전경아 옮김",인플루엔셜,2014,NaN,9788996991342,13180.0,189.200,"20,210",2015년 최다 판매 1위. 아들러 심리학에 관한 일본의 제1인자인 철학자 기시미 ...
...,...,...,...,...,...,...,...,...,...,...,...
995,996,감시와 처벌 :감옥의 역사,미셸 푸코 지음 ;오생근 옮김,나남출판,2004,NaN,9788930032483,94300.0,364.610,"1,832",미셸 푸코의 대표작. 현실을 보는 이 책의 입장은 부정적이다．마치 죄수들이 어디에 ...
996,997,괜찮은 사람 :강화길 소설,지은이: 강화길,문학동네,2016,NaN,9788954643023,3810.0,813.700,"1,830",일상에서 감지되는 불안의 기원을 천착하는 신인작가 강화길의 첫 소설집. 강화길은 8...
997,998,내가 죽으면 장례식에 누가 와줄까,김상현 (지은이),필름(Feelm),2019,NaN,9791188469321,NaN,818.000,"1,827",불행과 불행 사이에 끼어 있는 행복들을 마주하다!글로써 세상에 따스한 온기를 전하고...
998,999,면접이 막막할 때마다 꺼내 읽는 책 :면접관이 당신에게 원하는 98가지 대답,제임스 리드 지음 ;문수민 옮김,한빛비즈,2015,NaN,9791157840205,13320.0,325.337,"1,826",면접장에만 들어서면 자신감을 잃고 자책을 하는 이들을 위한 책. 세계 최대의 헤드헌...


In [27]:
df["DESCRIPTION"].replace('', np.nan, inplace=True)
df = df[df["DESCRIPTION"].notna()]
print('전체 문서의 수 :',len(df))

전체 문서의 수 : 997


/Users/yejinlee/opt/anaconda3/envs/practice_tensorflow/lib/python3.7/site-packages/pandas/core/series.py:4515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [28]:
corpus = []
for words in df['DESCRIPTION']:
  print(words)
  corpus.append(words.split())


2012년 12월 19일 국내 번역 출간된 이래 6년 연속 베스트셀러 순위 상위권을 차지하며 서점가에서 "21세기 가장 경이로운 베스트셀러"라고 불리는 소설. '2008~2017년, 지난 10년간 한국에서 가장 많이 팔린 소설', 히가시노 게이고의 대표작 &lt;나미야 잡화점의 기적&gt;이 국내 누적 판매 100만 부를 돌파했다.
2016년 맨부커상 수상작. 10년 전 작가 한강은 '내 여자의 열매'라는 단편소설을 썼다. 한 여자가 아파트 베란다에서 식물이 되고, 함께 살던 남자는 그녀를 화분에 심는 이야기였다. &lt;채식주의자&gt;는 언젠가 그 변주를 쓰고 싶다는 바람에서 출발했다. 표제작 '채식주의자', 2005년 이상문학상 수상작인 '몽고반점', 그리고 '나무 불꽃', 세 편의 중편으로 구성된 연작 소설로, 상처받은 영혼의 고통과 식물적인 상상력이 결합해 섬뜩한 아름다움을 뿜어낸다.
오늘의 젊은 작가 13권. 조남주 장편소설. 시사 교양 프로그램에서 10년 동안 일한 방송 작가답게 서민들의 일상에서 발생하는 비극을 사실적이고 공감대 높은 스토리로 표현하는 데 특출 난 재능을 보이는 작가는 &lt;82년생 김지영&gt;에서 30대를 살고 있는 한국 여성들의 보편적인 일상을 완벽하게 재현한다.
섬세한 감수성과 치밀한 문장으로 인간 존재의 본질을 탐구해온 작가 한강의 여섯번째 장편소설. '상처의 구조에 대한 투시와 천착의 서사'를 통해 한강만이 풀어낼 수 있는 방식으로 1980년 5월을 새롭게 조명한다.
2015년 최다 판매 1위. 아들러 심리학에 관한 일본의 제1인자인 철학자 기시미 이치로의 명 해석과 베스트셀러 작가인 고가 후미타케의 맛깔스러운 글이 잘 결합되어 새로운 형식을 선보인다.
펴내는 작품마다 압도적인 서사와 폭발적인 이야기의 힘으로 많은 독자들의 뜨거운 사랑을 받아온 정유정이 전작 &lt;28&gt; 이후 3년 만에 장편소설 &lt;종의 기원&gt;으로 독자들을 찾았다. 작품 안에서 늘 허를 찌르는 반전을 선사했던 작가답게, 이번 작품에서 

In [29]:
#한국어 불용어 정의
stops = [
  '휴',
  '아',
  '아이구',
  '아이쿠',
  '아이고',
  '어',
  '나',
  '우리',
  '저희',
  '따라',
  '의해',
  '을',
  '를',
  '에',
  '의',
  '가',
  '으로',
  '로',
  '에게',
  '뿐이다',
  '의거하여',
  '근거하여',
  '입각하여',
  '기준으로',
  '예하면',
  '저',
  '소인',
  '소생',
  '저희',
  '지말고',
  '하지마',
  '하지마라',
  '다른',
  '물론',
  '또한',
  '그리고',
  '막론하고',
  '관계없이',
  '그러나',
  '그런데',
  '하지만',
  '든간에',
  '설사',
  '비록',
  '더라도',
  '불문하고',
  '향하여',
  '향해서',
  '향하다',
  '쪽으로',
  '틈타',
  '이용하여',
  '타다',
  '오르다',
  '제외하고',
  '하여야',
  '비로소',
  '외에도',
  '이곳',
  '여기',
  '부터',
  '기점으로',
  '따라서',
  '하려고하다',
  '이리하여',
  '그리하여',
  '하지만',
  '일때',
  '할때',
  '앞에서',
  '중에서',
  '보는데서',
  '으로써',
  '로써',
  '까지',
  '해야한다',
  '일것이다',
  '반드시',
  '할줄알다',
  '할수있다',
  '할수있어',
  '한다면',
  '등',
  '등등',
  '제',
  '겨우',
  '단지',
  '다만',
  '할뿐',
  '딩동',
  '댕그',
  '대해서',
  '대하여',
  '대하면',
  '훨씬',
  '얼마나',
  '얼마만큼',
  '얼마큼',
  '남짓',
  '여',
  '얼마간',
  '약간',
  '다소',
  '좀',
  '조금',
  '다수',
  '몇',
  '얼마',
  '지만',
  '하물며',
  '또한',
  '그러나',
  '그렇지만',
  '하지만',
  '이외에도',
  '뿐이다',
  '다음에',
  '반대로',
  '만약',
  '그렇지않으면',
  '까악',
  '툭',
  '딱',
  '삐걱거리다',
  '보드득',
  '비걱거리다',
  '꽈당',
  '응당',
  '해야한다',
  '각',
  '각각',
  '여러분',
  '각종',
  '각자',
  '제각기',
  '하도록하다',
  '와',
  '과',
  '그러므로',
  '그래서',
  '고로',
  '거니와',
  '이지만',
  '대하여',
  '관하여',
  '관한',
  '과연',
  '실로',
  '아니나다를가',
  '생각한대로',
  '진짜로',
  '한적이있다',
  '하곤하였다',
  '하',
  '하하',
  '허허',
  '아하',
  '거바',
  '와',
  '오',
  '왜',
  '어째서',
  '무엇때문에',
  '어찌',
  '하겠는가',
  '무슨',
  '어디',
  '어느곳',
  '더군다나',
  '하물며',
  '더욱이는',
  '어느때',
  '언제',
  '야',
  '이봐',
  '어이',
  '여보시오',
  '흐흐',
  '흥',
  '휴',
  '헉헉',
  '헐떡헐떡',
  '영차',
  '여차',
  '어기여차',
  '끙끙',
  '아야',
  '앗',
  '아야',
  '콸콸',
  '졸졸',
  '좍좍',
  '뚝뚝',
  '주룩주룩',
  '솨',
  '우르르',
  '그래도',
  '또',
  '그리고',
  '바꾸어말하면',
  '바꾸어말하자면',
  '혹은',
  '혹시',
  '답다',
  '및',
  '그에 따르는',
  '때가 되어',
  '즉',
  '지든지',
  '설령',
  '가령',
  '하더라도',
  '할지라도',
  '일지라도',
  '지든지',
  '몇',
  '거의',
  '하마터면',
  '인젠',
  '이젠',
  '된바에야',
  '된이상',
  '그위에',
  '게다가',
  '고려하면',
  '하게될것이다',
  '일것이다',
  '비교적',
  '좀',
  '보다더',
  '비하면',
  '시키다',
  '하게하다',
  '할만하다',
  '의해서',
  '연이서',
  '이어서',
  '잇따라',
  '뒤따라',
  '뒤이어',
  '결국',
  '의지하여',
  '기대여',
  '통하여',
  '자마자',
  '더욱더',
  '불구하고',
  '얼마든지',
  '마음대로',
  '곧',
  '즉시',
  '바로',
  '당장',
  '하자마자',
  '하면된다',
  '그래',
  '그렇지',
  '요컨대',
  '즉',
  '구체적으로',
  '말하자면',
  '시작하여',
  '시초에',
  '이상',
  '허',
  '헉',
  '허걱',
  '바와같이',
  '해도좋다',
  '해도된다',
  '게다가',
  '더구나',
  '하물며',
  '와르르',
  '팍',
  '퍽',
  '펄렁',
  '동안',
  '이래',
  '하고있었다',
  '이었다',
  '에서',
  '로부터',
  '까지',
  '예하면',
  '했어요',
  '해요',
  '함께',
  '같이',
  '더불어',
  '마저',
  '마저도',
  '양자',
  '모두',
  '습니다',
  '가까스로',
  '하려고하다',
  '즈음하여',
  '다른',
  '해봐요',
  '습니까',
  '했어요',
  '무릎쓰고',
  '개의치않고',
  '매',
  '매번',
  '들',
  '모',
  '어느것',
  '어느',
  '로써',
  '갖고말하자면',
  '어디',
  '어느쪽',
  '어느것',
  '어느해',
  '언젠가',
  '어떤것',
  '어느것',
  '저기',
  '저쪽',
  '저것',
  '그때',
  '그럼',
  '그러면',
  '요만한걸',
  '그래',
  '그때',
  '저것만큼',
  '그저',
  '이르기까지',
  '너',
  '너희',
  '당신',
  '어찌',
  '설마',
  '차라리',
  '할지언정',
  '할지라도',
  '할망정',
  '할지언정',
  '구토하다',
  '게우다',
  '토하다',
  '메쓰겁다',
  '옆사람',
  '퉤',
  '쳇',
  '의거하여',
  '근거하여',
  '의해',
  '따라',
  '힘입어',
  '그',
  '다음',
  '버금',
  '두번째로',
  '기타',
  '첫번째로',
  '나머지는',
  '그중에서',
  '견지에서',
  '입장에서',
  '위해서',
  '단지',
  '의해되다',
  '하도록시키다',
  '뿐만아니라',
  '반대로',
  '전후',
  '전자',
  '앞의것',
  '잠시',
  '잠깐',
  '하면서',
  '그렇지만',
  '다음에',
  '그러한즉',
  '그런즉',
  '남들',
  '아무거나',
  '어찌하든지',
  '같다',
  '비슷하다',
  '예컨대',
  '이럴정도로',
  '어떻게',
  '만약',
  '만일',
  '만약에',
  '무엇',
  '무슨',
  '어느',
  '어떤',
  '아래윗',
  '조차',
  '한데',
  '여전히',
  '심지어',
  '까지도',
  '조차도',
  '때',
  '시각',
  '무렵',
  '시간',
  '동안',
  '어때',
  '어떠한',
  '하여금',
  '네',
  '예',
  '우선',
  '누구',
  '그러니',
  '그러니까',
  '때문에',
  '그',
  '너희',
  '그들',
  '너희들',
  '타인',
  '것',
  '것들',
  '너',
  '위하여',
  '공동으로',
  '동시에',
  '어찌하여',
  '무엇때문에',
  '붕붕',
  '윙윙',
  '나',
  '우리',
  '엉엉',
  '휘익',
  '윙윙',
  '오호',
  '아하',
  '어쨋든',
  '차라리',
  '흐흐',
  '놀라다',
  '마치',
  '아니라면',
  '쉿',
  '아니었다면',
  '하든지',
  '아니면',
  '이라면',
  '좋아',
  '알았어',
  '하는것도',
  '그만이다',
  '하나',
  '일',
  '일반적으로',
  '일단',
  '한켠으로는',
  '오자마자',
  '이렇게되면',
  '이와같다면',
  '전부',
  '한마디',
  '한항목',
  '근거로',
  '하기에',
  '아울러',
  '이르기까지',
  '까닭으로',
  '이유만으로',
  '그래서',
  '그러므로',
  '있다',
  '어떤것',
  '연관되다',
  '어떤것들',
  '이리하여',
  '그리하여',
  '여부',
  '하기보다는',
  '하느니',
  '운운',
  '이러이러하다',
  '하구나',
  '하도다',
  '다시말하면',
  '다음으로',
  '우리',
  '우리들',
  '오히려',
  '하기는한데',
  '어떻게',
  '어떻해',
  '어찌됏어',
  '어때',
  '어째서',
  '본대로',
  '자',
  '이',
  '이쪽',
  '여기',
  '이것',
  '이번',
  '이렇게말하자면',
  '이런',
  '이러한',
  '이만큼',
  '이때',
  '이렇구나',
  '끼익',
  '삐걱',
  '따위',
  '왜냐하면',
  '중의하나',
  '오직',
  '오로지',
  '도착하다',
  '도달하다',
  '관해서는',
  '여러분',
  '혼자',
  '자기',
  '자기집',
  '자신',
  '총적으로',
  '으로서',
  '참',
  '그만이다',
  '쿵',
  '탕탕',
  '쾅쾅',
  '둥둥',
  '봐',
  '봐라',
  '아이야',
  '아니',
  '와아',
  '응',
  '아이',
  '참나',
  '년',
  '월',
  '일',
  '령',
  '영',
  '일',
  '이',
  '삼',
  '사',
  '오',
  '육',
  '륙',
  '칠',
  '팔',
  '구',
  '이천육',
  '이천칠',
  '이천팔',
  '이천구',
  '하나',
  '둘',
  '셋',
  '넷',
  '다섯',
  '여섯',
  '일곱',
  '여덟',
  '아홉',
  '령',
  '영',
]

In [32]:
glove_embedding = pd.read_fwf("./glove.txt",header=None, names= ["Vector", "Word"])

In [33]:
first_words = []
Vectors = []
for i in glove_embedding[:]['Vector']:
    first_word = i.split()[0]
    first_words.append(first_word)
    Vectors.append(i.split(' ', 1)[1])
glove_embedding['Word'] = first_words
glove_embedding['Vector'] = Vectors
#사전 훈련된 글러브 워드 임베딩 형식이 "단어 0.12 0.23 -0.245 ..." 이런식으로 되있기 때문에 앞에 있는 단어들을 모으고 벡터 숫자들을 따로 모았습니다.
#이렇게 모아놓으면, 책들의 줄거리들을 비교할때 쉽게 단어들을 찾을 수 있습니다.

In [34]:
print(first_words [0])


1


In [35]:
#이 함수는 각 책의 줄거리를 벡터화 시키는 과정입니다!
#줄거리에 있는 단어들을 워드 임베딩에서 찾아내는 중입니다.
def vectors(document_list):
    document_embedding_list = []

    # 각 문서에 대해서
    for line in document_list:
        doc2vec = None
        count = 0
        #여기서 전처리 작업을 진핸합니다.
        line = re.sub('&.t;', '', line)
        #이상한 기호들을 없애는 중입니다!
        corpus = [w for w in line.split() if not w in stops]
        #아래 코드는 각 단어에 해당하는 벡터를 찾아 문서에 추가하는 과정입니다.
        for word in corpus:
            if word in first_words:
                index = first_words.index(word)
                count += 1
                if doc2vec is None:
                    doc2vec = Vectors[index]
                else:
                    doc2vec = doc2vec + Vectors[index]
                    
        if doc2vec is not None:
            doc2vec_list = np.asarray(doc2vec.split())
            doc2vec_final_list = []
            for i in doc2vec_list:
               
                if len(i) > 10:
                    checked = 1
                    for idx, val in enumerate(i):
                        if idx != 0 and val == "-" and checked !=0 :
                            checked = 0
                            doc2vec_final_list.append(i[0:idx])
                            doc2vec_final_list.append(i[idx:len(i)])
                        elif idx !=1 and idx !=2 and val == "." and checked !=0 :
                            checked = 0
                            doc2vec_final_list.append(i[0:idx-1])
                            doc2vec_final_list.append(i[idx-1:len(i)])
                        
                else:
                    if i[:2]=='0-' or i[:2]=='--':
                        doc2vec_final_list.append(i[1:])
                    else:
                        doc2vec_final_list.append(i)
            doc2vec_list = list(map(float, doc2vec_final_list))
            doc2vec = np.asarray(doc2vec_list) / count
            doc2vec.resize(1300)
            document_embedding_list.append(doc2vec)

    # 각 문서에 대한 문서 벡터 리스트를 리턴
    return document_embedding_list

In [38]:
document_embedding_list = vectors(df['DESCRIPTION'])

In [37]:
print('문서 벡터의 수 :',len(document_embedding_list))

문서 벡터의 수 : 997


In [39]:
from numpy import dot
from numpy.linalg import norm

#두개의 책들을 비교할때, 두 책들의 백터를 비교하면서 얼마나 비슷한 책인지 찾는 함수입니다.
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [40]:
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)

In [41]:
sim_scores = list(enumerate(cosine_similarities[1]))

In [42]:
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

In [80]:
#채식주의자:한강 연작소설을 읽고 유사한 책 추천 받기
#채식주의자 index_num = 1

index_num = 1 #어딴 책과 유사한 책을 추천받고 싶나요? 그 책의 index를 입력해주세요!:)


print(df["서명"][index_num] +"과 유사한 책을 추천해드릴게요:) 🔥")

print("-------------------------------------------------------\n\n")

sim_scores = list(enumerate(cosine_similarities[index_num]))

#cosine유사도 비슷한 순으로 재정렬
sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)

for idx, val in enumerate(sim_scores[1:6]):
    print(str(idx +1) + '.' + df["서명"][val[0]])

채식주의자:한강 연작소설과 유사한 책을 추천해드릴게요:) 🔥
-------------------------------------------------------


1.희랍어 시간 :한강 장편소설 
2.단 하루도 너를 사랑하지 않은 날이 없다 
3.파리 5구의 여인 :더글라스 케네디 장편소설 
4.사랑의 온도 :착한 스프는 전화를 받지 않는다 
5.나를 사랑하지 않는 나에게 :존중받지 못한 내 마음을 위한 심리학 
